# Hyperparameter Tuning using HyperDrive

In [1]:
import os
import csv

from sklearn import datasets
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.widgets import RunDetails

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive import normal, choice

## Initialize Workspace and Experiment

In [2]:
project_folder = './hyper_drive_proj'
ws = Workspace.from_config()
experiment_name = 'hyperDrive-experiment'
experiment=Experiment(ws, experiment_name)

## Dataset
Loading dataset is done inside train.py script.

## Creating Compute Target

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "demo-cluster"
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 1)

Found existing cluster, use it.
Succeeded............
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Hyperdrive Configuration
Model params and configuration is explained in readme file.

In [40]:
early_termination_policy = BanditPolicy(evaluation_interval=3, slack_factor=0.1, delay_evaluation=3)
param_sampling = RandomParameterSampling({
    'C': choice(0.01, 0.1, 1),
    '--max_iter': choice(2, 5, 10)
})

est = SKLearn(source_directory='.', compute_target=compute_target, entry_script = 'train.py', script_params={'--C':1.0, '--max_iter':100})

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                     primary_metric_name='Accuracy', 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=2, 
                                     max_concurrent_runs=4, 
                                     policy=early_termination_policy,
                                     estimator=est)


In [43]:
hyperdrive_run = experiment.submit(hyperdrive_config)

## Run Details

In [44]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [45]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_c840f415-2d66-4545-a836-ac7844254385
Web View: https://ml.azure.com/runs/HD_c840f415-2d66-4545-a836-ac7844254385?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-165055/workspaces/quick-starts-ws-165055&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_c840f415-2d66-4545-a836-ac7844254385
Web View: https://ml.azure.com/runs/HD_c840f415-2d66-4545-a836-ac7844254385?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-165055/workspaces/quick-starts-ws-165055&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_c840f415-2d66-4545-a836-ac7844254385',
 'target': 'demo-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-12-05T09:40:19.596931Z',
 'endTimeUtc': '2021-12-05T09:43:51.734454Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '951b7413-3be2-40af-9aa8-ff2d08791c9e',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1056-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.34.0',
  'score': '0.8792535675082327',
  'best_child_run_id': 'HD_c840f415-2d66-4545-a836-ac7844254385_0',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg165055.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_c840f415-2d66-4545-a836-ac7844254385/azureml-logs/hyperdrive.txt?sv

## Best Model

In [47]:
# get the best run and display the properties of the model
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength:',best_run_metrics['Regularization Strength:'])
print('\n Max iterations:',best_run_metrics['Max iterations:'])

Best Run Id:  HD_c840f415-2d66-4545-a836-ac7844254385_0

 Accuracy: 0.8792535675082327

 Regularization Strength: 0.01

 Max iterations: 10


In [48]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
hyperDrive-experiment,HD_c840f415-2d66-4545-a836-ac7844254385_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [53]:
import joblib
best_run.download_file(name='model.joblib', output_file_path='.outputs')
model = best_run.register_model(model_name='hyperdrive_run', model_path='model.joblib')

UserErrorException: UserErrorException:
	Message: File with path model.joblib was not found,
available files include: logs/azureml/22_azureml.log,outputs/model.pkl,system_logs/cs_capability/cs-capability.log,system_logs/hosttools_capability/hosttools-capability.log,system_logs/lifecycler/execution-wrapper.log,system_logs/lifecycler/lifecycler.log,system_logs/lifecycler/vm-bootstrapper.log,user_logs/std_log.txt.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "File with path model.joblib was not found,\navailable files include: logs/azureml/22_azureml.log,outputs/model.pkl,system_logs/cs_capability/cs-capability.log,system_logs/hosttools_capability/hosttools-capability.log,system_logs/lifecycler/execution-wrapper.log,system_logs/lifecycler/lifecycler.log,system_logs/lifecycler/vm-bootstrapper.log,user_logs/std_log.txt."
    }
}

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

### NOTE: Since the AutoMl model is performing better and since we have to deploy only one model, skipping hyperDrive model deployment step as I already deployed Automl model.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

